# Subsoil schematisation and parametrisation

In order to run the ROSE model it is necessary to have information about the subsoil. 

For the analysis of railway lines the build-up of the subsoil plays a critical role on the behaviour of the infrastructure. Railway tracks cover large lenghts of different subsoil conditions, therefore it is necessary to take into account the subsoil spatial variability.

In ROSE the subsoil and the effect of its variability on the response of railway tracks, has been accounted for by means of a stochastic subsoil model. The stochastic subsoil model defines build-up scenario’s that are constructedby combining subsoil information (such as in situ and laboratory geotechnical testing) with geological knowledge. This approach has been derived from the guidelines to assess and design flood defences in the Netherlands.

The SOS is available for the entire A2 corridor. The SOS is being extended to cover the entire ProRail network.

## Create the SOS for the A2 corridor
The processing of the SOS is available under data_proc

In [ ]:
import numpy as np
from data_proc import SoS

In [ ]:
sos = SoS.ReadSosScenarios("../data/SoS/soilprofiles.csv",
                           "../data/SoS/20201102_Prorail_parameters_SOS.csv",
                           "../data/SoS/segments.csv",
                           "../data/SoS/Segments_TKI_v2.shp"
                          )

In [ ]:
sos.SOS

In [ ]:
sos.create_segments()

In [ ]:
sos.SOS

In [ ]:
sos.dump("./results/sos.json")

In [ ]:
sos.plot_sos(output_folder="./results/SOS_plots")

## Determine the subsoil stiffness and damping properties

The dynamic soil spring stiffness and dashpot damping, that are required for the dynamic train-track analyses, were computed by means of a semi-analytical cone model based on one-dimensional wave propagation. This model assumes that the load is applied to a disk at the surface, which induces stresses on an area that increases with depth; the displacements are constant at the cross section of the cone. Discontinuities can be modelled by assuming at the interface between two layers that one cone leads to the creation of two new cones: reflected and refracted. This allows the modelling of multi-layered soils. The cone method has been extensively applied, because it has the advantage of providing an accurate numerical solution with limited computational efforts. In ROSE, the dynamic stiffness, Kdyn and damping, D, follow the definition:

$$K_{dyn}=\Re \left(S\right)$$

$$D=\frac{\Im \left(S \right)}{\omega}$$

where S is the dynamic stiffness matrix (complex frequency response function) and ω the angular frequency.

### Define the ballast properties
The SOS only contains the subsoil. The ballast needs to be added

In [ ]:
G0 = 40e6
poisson = 0.2
density = 2000
damping = 0.05
thickness = 1
ballast = ["ballast", G0, poisson, density, damping, thickness]

In [ ]:
from run_rose import run_wolf

In [ ]:
new_soil = run_wolf.read_file(r"./results/sos.json", ballast)

In [ ]:

for soil in new_soil:
    print(soil[0])

In [ ]:
# run wolf for the first two segments : 15 files
omega = np.linspace(0, 314, 100)
run_wolf.run_wolf(new_soil[:15], omega, output=r"./results/dyn_stiffness", plots=True)

In [ ]:
import json
with open(r'./results/dyn_stiffness/Kdyn_Segment 1001_scenario 1.json', "r") as f:
    data = json.load(f)
print(data.keys())

In [ ]:
import matplotlib.pylab as plt
from data_proc import sensar
sensar_data = sensar.read_geopackage("../data/Sensar/data.gpkg")

## Update SOS with InSar data
InSar data is available for the A2 corridor (through Sensar).
In data_proc there are tools to read and process the InSar dataset.

In [ ]:
sensar_data["1"].keys()

In [ ]:
plt.plot(sensar_data["1"]["dates"], sensar_data["1"]["settlements"])

In [ ]:
sensar.save_sensar_data(sensar_data, "./results/settlements.pickle")

In [ ]:
name = "Segment 1030"
sos.SOS[name]

# get coordinates of current segments
coordinates = np.array(list(sos.SOS[name].values())[0]['coordinates'])

# get coordinate limits
xlim = [min(coordinates[:,0]), max(coordinates[:,0])]
ylim = [min(coordinates[:,1]), max(coordinates[:,1])]

# get date limits from sensar data and fugro data
sensar_dates = list(sensar_data.values())[0]["dates"]

# add plot of Sensar settlement measurements within the current segment
sensar_items_within_bounds = sensar.get_all_items_within_bounds(sensar_data, xlim, ylim)
if sensar_items_within_bounds:
    _, _ = sensar.plot_settlements_from_item_list_over_time(sensar_items_within_bounds,date_lim=[min(sensar_dates), max(sensar_dates)])
    plt.grid()

The data can be filtered using Kalman Filtering.


In [ ]:
sensar_filtered = sensar.filter_dataset(sensar_data)
sensar.save_sensar_data(sensar_filtered, "./results/settlements_filtered.pickle")

In [ ]:
name = "Segment 1030"
sos.SOS[name]

# get coordinates of current segments
coordinates = np.array(list(sos.SOS[name].values())[0]['coordinates'])

# get coordinate limits
xlim = [min(coordinates[:,0]), max(coordinates[:,0])]
ylim = [min(coordinates[:,1]), max(coordinates[:,1])]

# get date limits from sensar data and fugro data
sensar_dates = list(sensar_filtered.values())[0]["dates"]

# add plot of Sensar settlement measurements within the current segment
sensar_items_within_bounds = sensar.get_all_items_within_bounds(sensar_filtered, xlim, ylim)
if sensar_items_within_bounds:
    _, _ = sensar.plot_settlements_from_item_list_over_time(sensar_items_within_bounds,date_lim=[min(sensar_dates), max(sensar_dates)])
    plt.grid()


### Clustering of data
The InSar data is used to perform clustering of the SOS types along the network.

The clustering is done by K-means.

<img src="static/k_means.png" width="600">

The clustering is performed at each SOS segment, with the InSar data that is within that SOS segment. The number of classes corresponds to the number of scenarios of the segment.


In [ ]:
from data_proc import cluster

cluster.main("./results/settlements.pickle", "./results/sos.json", "./results/clustering", coord=True, plot=True)

In [ ]:
cluster.main("./results/settlements_filtered.pickle", "./results/sos.json", "./results/clustering_filtered", coord=True, plot=True)